Kelompok 1 artificial intelligence

Anggota Kelompok:
1. Yosef Laskar Haryo Utomo
2. Daniel Chandra

Preprocessing Data

In [4]:
!pip install xgboost
!pip install spe2py
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
import spe2py as spe
from datetime import datetime, timedelta

import xgboost as xgb
import plotly.graph_objects as ply

In [28]:
from pathlib import Path
import pandas as pd
data_directory = Path('dataset/mix')
files = data_directory.glob('*.spe')
for file in files:
    df = pd.read_csv(file, index_col=0)

In [29]:
file

WindowsPath('dataset/mix/bMCA Spectrum_20220707_102321_MIX06_0005.SPE')

In [17]:
!pip install tensorflow
!pip install keras_tuner
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import mean_squared_logarithmic_error


In [30]:
TRAIN_DATA_PATH = os.path.join(current_dir, 'dataset', 'train')
TEST_DATA_PATH = os.path.join(current_dir, 'dataset', 'test')

TARGET_NAME = '$DATA'

NameError: name 'current_dir' is not defined

In [25]:
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data  = pd.read_csv(TEST_DATA_PATH)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\user\\Desktop\\Dataset\\TRAIN'

In [26]:
TRAIN_DATA_PATH
TEST_DATA_PATH

'C:\\Users\\user\\Desktop\\Dataset\\TEST'

In [27]:
x_train, y_train = train_data.drop(TARGET_NAME, axis=1, inplace=True), train_data[TARGET_NAME]
x_test, y_test = test_data.drop(TARGET_NAME, axis=1, inplace=True), test_data[TARGET_NAME]

NameError: name 'train_data' is not defined

In [ ]:
ef scale_datasets(x_train, x_test):
    
  standard_scaler = StandardScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [ ]:
import kerastuner as kt
msle = MeanSquaredLogarithmicError()


def build_model(hp):
  model = tf.keras.Sequential()
  
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))


  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='KB'
)

tuner.search(x_train_scaled, y_train, epochs=10, validation_split=0.2)

In [ ]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.build(x_train_scaled.shape)
best_model.summary()

best_model.fit(
    x_train_scaled, 
    y_train,
    epochs=10,
    batch_size=64
)